# NN2: Implementacja propagacji wstecznej błędu
Adrianna Grudzień 

### Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

### Wczytanie danych

In [188]:
sq_train = pd.read_csv("../mio1/regression/square-simple-training.csv", index_col=0)
sq_test = pd.read_csv("../mio1/regression/square-simple-test.csv", index_col=0)
x = np.transpose(np.matrix(sq_train['x']))
y_true = np.transpose(np.matrix(sq_test['y']))
# x = x[:10]
# y_true = y_true[:10]

### Funkcja aktywacji

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

### Definicja sieci MLP

-`np.multiply` mnoży wektory po każdym elemencie

In [203]:
class MLP:
    def __init__(self, input_layer_size=[100,1], layers_sizes=[100,3,1], activ_fun=sigmoid):
        self.input_layer_size = input_layer_size
        self.layers_sizes = layers_sizes # wszystkie warstwy poza pierwszą
        self.activ_fun = activ_fun
        self.number_of_layers = len(layers_sizes)+1 # liczba wszystkich warstw
        self._initialize_weights_and_biases()
        
    def _initialize_weights_and_biases(self, min_val=-0.5, max_val=0.5):
        self.weights = [np.random.uniform(min_val, max_val, 
                                          size=[self.input_layer_size[1], self.layers_sizes[0]])]
        self.biases = []   
        for i in range(len(self.layers_sizes)-1):
            self.weights.append(np.random.uniform(min_val, max_val, 
                                                  size=[self.layers_sizes[i], self.layers_sizes[i+1]]))
            self.biases.append(np.random.uniform(min_val, max_val, size=self.layers_sizes[i]))
        self.biases.append(np.random.uniform(min_val, max_val, 
                                             size=self.layers_sizes[len(self.layers_sizes)-1]))
        
    @staticmethod # wywołuje się bezpośrednio na obiekcie
    def mse(y_pred, y_true):
        """Funkcja kosztu C."""
        return np.mean(np.square(y_pred-y_true))
    @staticmethod
    def sigmoid_derivative(x):
        return np.exp(-x) / np.square(1 + np.exp(-x))
        
    def forward(self, x):
        self.active = [x] # wektor wartości aktywowanych - a^l
        self.inactive = [x] # nieaktywowane - z^l
        for i in range(len(self.weights)):
            z = np.sum(self.active[i] @ self.weights[i]) + self.biases[i]
            a = self.activ_fun(z)
            self.inactive.append(z)
            self.active.append(a)
        return self.active[-1]
    
    def backpropagate(self, y_pred, y_true):
        """y_pred ~= aL"""
        delta_biases = [np.zeros(b.shape) for b in self.biases]
        delta_weigths = [np.zeros(w.shape) for w in self.weights]
        
        print(y_pred.shape, y_true.shape)
        print((y_pred - y_true).shape, (MLP.sigmoid_derivative(self.inactive[-1])).shape)
        delta = np.multiply(y_pred - y_true, MLP.sigmoid_derivative(self.inactive[-1])).transpose() # dla ostatniej warstwy - L
        print(delta.shape)
        delta_biases[-1] = delta
        print(self.active[-2], delta.shape)
        delta_weigths[-1] = self.active[-2] @ delta.transpose()
        for l in range(self.number_of_layers-2, -1, -1): #dla warstw (L-1,...,0)
            delta = np.multiply(self.weights[l] @ delta, MLP.sigmoid_derivative(self.inactive[l]).transpose())
            delta_biases[l] = delta
            delta_weigths[l] = self.active[l-1].transpose() @ delta.transpose()
        
            
#         for i in delta_weigths:
#             print(i.shape)
#         print()
#         for i in self.inactive:
#             print(i.shape)
#         return (delta_weigths, delta_biases)
    
    def fit(self, x, y_true):
        y_pred = self.forward(x)
        self.backpropagate(y_pred, y_true)
        

In [204]:
mlp = MLP(layers_sizes=[3,2,1])
mlp.fit(x, y_true)

(1,) (100, 1)
(100, 1) (1,)
(1, 100)
[0.2268918  0.29008086] (1, 100)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 2)